# Sentiment Analysis with distilBERT

Following the [HuggingFace tutorial](https://huggingface.co/docs/transformers/en/tasks/sequence_classification) on text classification using [Goodreads data](https://www.kaggle.com/competitions/goodreads-books-reviews-290312)

In [46]:
import pandas as pd

from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from datasets import load_dataset

In [48]:
df = pd.read_csv("data/goodreads_train.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/goodreads_train.csv'

In [8]:
df.describe()

,book_id,rating,n_votes,n_comments
count,9.000000e+05,900000.000000,900000.000000,900000.000000
mean,1.344145e+07,3.689639,3.294381,1.016656
std,9.357863e+06,1.252583,17.873553,5.963821
min,1.000000e+00,0.000000,-3.000000,-1.000000
25%,6.340471e+06,3.000000,0.000000,0.000000
50%,1.344203e+07,4.000000,0.000000,0.000000
75%,2.057897e+07,5.000000,2.000000,0.000000
max,3.632868e+07,5.000000,3222.000000,1335.000000


In [9]:
# transform the rating column to a label column

def rating_to_sent(rating):
    # rating 3 is assigned positive because sampling a few reviews, 
    # I personally would call most of them more positive than negative. This could be changed
    if rating > 3:
        return 1
    elif rating == 3:
        return 1
    elif rating < 3:
        return 0

df['label'] = df['rating'].apply(rating_to_sent)
df.head()

In [11]:
# rename column
df.rename(columns={'review_text':'text'}, inplace=True)
df.head()

,user_id,book_id,review_id,rating,text,date_added,date_updated,read_at,started_at,n_votes,n_comments,label
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0,1
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0,1
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,NaN,NaN,5,1,0
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1,1


In [30]:
hf_df = df[['text', 'label']]
hf_df.head()

,text,label
0,This is a special book. It started slow for ab...,1
1,Recommended by Don Katz. Avail for free in Dec...,1
2,"A fun, fast paced science fiction thriller. I ...",1
3,Recommended reading to understand what is goin...,0
4,"I really enjoyed this book, and there is a lot...",1


In [32]:
hf_df.to_csv("data/goodreads_for_hf.csv")

In [36]:
gr_dataset = load_dataset("csv", data_files="data/goodreads_for_hf.csv")

Generating train split: 0 examples [00:00, ? examples/s]

## Preprocessing

In [20]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")


In [38]:
#truncate input to the maximum length of BERT's input length

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation = True)

In [40]:
tokenized_gr = gr_dataset.map(preprocess_function, batched = True)

Map:   0%|          | 0/900000 [00:00<?, ? examples/s]

In [42]:
type(tokenized_gr)

datasets.dataset_dict.DatasetDict

In [44]:
# now pad the inputs that are too short
# tbh I know why to do this, but this function is a total black box at this point

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

NameError: name 'DataCollatorWithPadding' is not defined